<a href="https://colab.research.google.com/github/julianflowers/herbivores_ghg/blob/master/unified_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
install.packages("remotes")
remotes::install_github("kth-library/semanticscholar", dependencies = TRUE)

(as ‘lib’ is unspecified)







	‘/tmp/RtmpLlHL3U/downloaded_packages’

  Use `force = TRUE` to force installation



In [3]:
%%R
library(semanticscholar)

In [4]:
%%R
search <- "herbivory climate change"
s2_search <- semanticscholar::S2_search_papers(keyword = search, offset = 1, limit = 100)
s2_search

$total
[1] 617520

$offset
[1] 1

$`next`
[1] 101

$data
# A tibble: 100 × 2
   paperId                                  title                               
 * <chr>                                    <chr>                               
 1 a9ce39a660c48a3602c00296d5d8f24ea14ecb10 Potential of Climate Change and Her…
 2 16e220c74ab95d69c96a34e1cd79adeb03390bce Asymmetric Responses to Climate Cha…
 3 44ab5396ac4ef4dd865f8c1875ee27f46f68fcc9 Potential for an Impact of Global C…
 4 a90799b9e3fe74b7e506503be8ad647038e87e46 Long-term changes in the primary pr…
 5 c17b737748a922c5339873e2720a89520814331c Cloud cover and delayed herbivory r…
 6 1bcfc008e53548f3271a64d2f5f2b59ef6dd33f6 Circumpolar arctic tundra biomass a…
 7 5880b50f3c06e61056cb94ec1e636a67cada44d8 Physiological and biochemical respo…
 8 88caa8935016636c578bf00f8e57b5c8950e6afc Climate change alters plant-herbivo…
 9 78e45f7368cb7f344b4a11117b92b6fce80621dd Synergistic effects of insect herbi…
10 1ab15bbfc0c912fcf805ceb5b60c6

In [5]:
%%R
library(tidyverse)


✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [6]:
%%R

ids <- pluck(s2_search$data, "paperId")

papers <- map(ids, S2_paper)

papid <- map(papers, "paperId") |>
  enframe()

abstract <- map(papers, "abstract") |>
  enframe()

topics <- map(papers, "topics") |>
  enframe()

doi <- map(papers, "doi") |>
  enframe()

title <- map(papers, "title") |>
  enframe()

corpusId <- map(papers, "corpusId") |>
  enframe()

fos <- map(papers, "fieldsOfStudy") |>
  enframe()

combined <- papid |>
  left_join(title, by = "name") |>
  left_join(abstract, by = "name") |>
  left_join(fos, by = "name") |>
  left_join(topics, by = "name") |>
  left_join(corpusId, by = "name") |>
  left_join(doi, by = "name")

combined <- combined |> 
  rename(id = value.x, 
         title = value.y, 
         abstract = value.x.x, 
         fos = value.y.y, 
         topics = value.x.x.x, 
         corpusId = value.y.y.y, 
         doi = value)


In [7]:
%%R
comb <- combined |>
  unnest(cols = "topics") |>
  unnest(cols = "fos") |>
  group_by(name) |>
  select(-c(topicId, url)) |>
  summarise(topic = paste(topic, collapse = "; ")) 

  comb

# A tibble: 44 × 2
    name topic                                                                  
   <int> <chr>                                                                  
 1     2 Herbivory; Herbivory; GLUCOSE OXIDASE; GLUCOSE OXIDASE; Lycopersicon e…
 2     4 Reindeer; Herbivory; Labrador retriever; Population; Interaction; Habi…
 3     5 Onset (audio); Onset (audio); Ephrin Type-B Receptor 1, human; Ephrin …
 4     6 Herbivory; Herbivory; Arctic Tundra; Arctic Tundra; Climate Change; Cl…
 5     7 Carbon Dioxide; Carbon Dioxide; Sea urchin (invertebrate); Sea urchin …
 6    11 Herbivory                                                              
 7    15 Plant Development; Plant Development; Herbivory; Herbivory; Pinus <gen…
 8    21 Tea; Carbon Dioxide; Climate Change; Herbivory; Other Toxicity Studies…
 9    22 Coral Reefs; Coral Reefs; Fishes; Fishes; Herbivory; Herbivory; Seawee…
10    23 Herbivory; Herbivory; Trees (plant); Trees (plant); Gradient; Gradient…
# … with 

In [8]:
%%R
comb |>
  write_delim("test.txt")
  

In [9]:
%%R
abstracts <- combined |>
    select(id, abstract)
abstracts |>
    unnest(col = c("abstract")) |>
    write_delim("abstract.txt")

In [10]:
%pip install taxonerd
%pip install https://github.com/nleguillarme/taxonerd/releases/download/v1.3.0/en_ner_eco_biobert-1.0.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.0.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.8 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.4
    Uninstalling spacy-3.2.4:
      Successfully uninstalled spacy-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.0 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.0.8 which is incompatible.
en-core-sci-sm 0.5.0 requires spacy<3.3.0,>=3.2.3, but you have spacy 3.0.8 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 402.1 MB 26 kB/s 


In [11]:
from taxonerd import TaxoNERD

In [12]:
ner = TaxoNERD(model="en_ner_eco_biobert", prefer_gpu=True, with_linking="taxref", with_abbrev=True) # Add with_linking="gbif_backbone" or with_linking="taxref" to activate entity linking


In [13]:
doc = ner.find_in_file("/content/test.txt")


In [14]:
doc

,offsets,text,entity
T0,LIVB 70 93,Lycopersicon esculentum,"[(TAXREF:124070, Lycopersicon esculentum, 1.0)]"
T1,LIVB 95 118,Lycopersicon esculentum,"[(TAXREF:124070, Lycopersicon esculentum, 1.0)]"
T2,LIVB 162 177,Helicoverpa zea,"[(TAXREF:646124, Helicoverpa zea, 1.0)]"
T3,LIVB 179 194,Helicoverpa zea,"[(TAXREF:646124, Helicoverpa zea, 1.0)]"
T4,LIVB 409 417,Reindeer,"[(TAXREF:61064, Reindeer, 1.0)]"
...,...,...,...
T60,LIVB 9607 9612,Pinus,"[(TAXREF:196293, Pinus, 0.9999999403953552)]"
T61,LIVB 9622 9633,Pinus taeda,"[(TAXREF:446381, Pinus taeda, 1.0)]"
T62,LIVB 9780 9792,Pinus edulis,"[(TAXREF:717410, Pinus edulis, 1.0)]"
T63,LIVB 12064 12085,Plesiastrea versipora,"[(TAXREF:445226, Plesiastrea versipora, 1.0)]"


In [15]:
doc1 = ner.find_in_file("/content/abstract.txt")
doc1

,offsets,text,entity
T0,LIVB 1632 1647,Geometrid moths,"[(TAXREF:185259, geometer moths, 0.80506289005..."
T1,LIVB 5392 5400,Rangifer,"[(TAXREF:197043, Rangifer, 1.0)]"
T2,LIVB 6123 6131,Rangifer,"[(TAXREF:197043, Rangifer, 1.0)]"
T3,LIVB 8163 8169,Insect,"[(TAXREF:51868, Water Stick Insect, 0.70065522..."
T4,LIVB 8735 8746,dwarf birch,"[(TAXREF:762263, Dwarf birch, 1.0)]"
...,...,...,...
T230,LIVB 137155 137164,sea hares,"[(TAXREF:188096, sea hares, 1.0)]"
T231,LIVB 143143 143154,butterflies,"[(TAXREF:185214, butterflies and moths, 0.7505..."
T232,LIVB 143159 143164,moths,"[(TAXREF:185244, case moths, 0.8185053467750549)]"
T233,LIVB 144272 144295,Ambrosia artemisiifolia,"[(TAXREF:82080, Ambrosia artemisiifolia, 1.0)]"


In [ ]:
!pip install flair
!pip install -U spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz


import flair
from flair.data import Sentence
from flair.models import SequenceTagger
import spacy
import scispacy
from flair.models import MultiTagger
from flair.tokenization import SciSpacyTokenizer

In [17]:
tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")


2022-08-12 11:55:45,909 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1
2022-08-12 11:56:24,180 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [18]:
import pandas as pd
dataset=pd.read_csv("/content/abstract.txt",delimiter="\t")

dataset

,id abstract
0,"""Compared to most other forest ecosystems, ci..."
1,"""Rapid warming in northern ecosystems over th..."
2,"""Satellite remote sensing data have indicated..."
3,"""Plant-herbivore interactions have evolved in..."
4,"""Climate change increases the insect abundanc..."
...,...
96,CONCLUSIONS
97,Range-expansion by plants may result in above-...
98,"""Abstract This chapter discusses the abiotic ..."
99,"""Predicting how ecological interactions will ..."


In [19]:
dataset["id abstract"]
b = dataset['id abstract']
b

0       "Compared to most other forest ecosystems, ci...
1       "Rapid warming in northern ecosystems over th...
2       "Satellite remote sensing data have indicated...
3       "Plant-herbivore interactions have evolved in...
4       "Climate change increases the insect abundanc...
                             ...                        
96                                           CONCLUSIONS
97     Range-expansion by plants may result in above-...
98      "Abstract This chapter discusses the abiotic ...
99      "Predicting how ecological interactions will ...
100     "Abstract Climate change may affect plant–her...
Name: id abstract, Length: 101, dtype: object

In [49]:
s = b[4]

In [50]:
text = Sentence(s)

ner = tagger.predict(text)

In [39]:
tagger1 = MultiTagger.load("hunflair")

2022-08-12 14:19:51,133 loading file /root/.flair/models/hunflair-celline-v1.0.pt
2022-08-12 14:20:10,385 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-CellLine, B-CellLine, I-CellLine, E-CellLine, <START>, <STOP>
2022-08-12 14:20:10,836 loading file /root/.flair/models/hunflair-chemical-full-v1.0.pt
2022-08-12 14:20:29,711 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-Chemical, B-Chemical, I-Chemical, E-Chemical, <START>, <STOP>
2022-08-12 14:20:30,135 loading file /root/.flair/models/hunflair-disease-full-v1.0.pt
2022-08-12 14:20:44,511 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, B-Disease, E-Disease, I-Disease, S-Disease, <START>, <STOP>
2022-08-12 14:20:44,971 loading file /root/.flair/models/hunflair-gene-full-v1.0.pt
2022-08-12 14:21:06,095 SequenceTagger predicts: Dictionary with 8 tags: <unk>, O, S-Gene, B-Gene, I-Gene, E-Gene, <START>, <STOP>
2022-08-12 14:21:06,619 loading file /root/.flair/models/hunflair-species-full-v1.1.pt
2022

In [51]:
tagger1.predict(text)

for annotation_layer in text.annotation_layers.keys():
    for entity in text.get_spans(annotation_layer):
        print(entity)

Span[160:161]: "sesquiterpene" → Chemical (0.8883)
Span[162:163]: "homoterpene" → Chemical (0.9141)
Span[107:109]: "dwarf birch" → Species (0.7591)
Span[110:112]: "Betula nana" → Species (0.9611)
Span[124:126]: "autumnal moth" → Species (0.881)
Span[127:129]: "Epirrita autumnata" → Species (0.9454)
Span[132:134]: "winter moth" → Species (0.8626)
Span[135:137]: "Operophtera brumata" → Species (0.8825)
Span[144:146]: "B. nana" → Species (0.7793)


In [52]:
print('The following NER tags are found:')
# iterate over entities and print
for entity in text.get_spans('ner'):
    print(entity)

The following NER tags are found:
